In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoCV
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
import itertools

/Users/lperez/.local/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Simplifying the model
From the EDA phase we concluded that both `region` and `children` are hardly relevant predictors. Therefore we migh just drop it from our data

In [2]:
csvFilePath = './datasets_13720_18513_insurance.csv'
with open (csvFilePath, 'rb') as file:
    data = pd.read_csv(file, encoding = 'UTF-8',
                                    thousands = ',',
                                    decimal = '.',
                                    dtype = {
                                            'sex':'category',
                                            'smoker':'category',
                                            'region':'category',
                                            'children':'category',
                                            }
                                )

cols_to_drop = ['region', 'children']
data = data.drop(cols_to_drop, axis=1)



def gen_dummy_col_names(dataframe):
    category_dataframe = dataframe.select_dtypes(include='category')
    dummy_col_names = []
    for c in category_dataframe.columns:
        unique_col_vals = category_dataframe[c].unique().tolist()
        dummy_col_names.append([c + '_' + val for val in unique_col_vals])
    
    return dummy_col_names

def gen_dummy_cols_inner_combinations(dummy_col_names):
    #dummy_col_names should be a list of lists, ie
    #[['sex_female', 'sex_male'], ['smoker_yes', 'smoker_no']]
    dummy_cols_inner_combinations = []
    for col_list in dummy_col_names:
        col_list.sort()
        list_lenght = len(col_list)
        if list_lenght > 1:
            for lenght in range(2, list_lenght+1):
                for subset in itertools.combinations(col_list, lenght):
                    dummy_cols_inner_combinations.append(subset)
    
    dummy_cols_inner_combinations = [list(element) for element in dummy_cols_inner_combinations]
    dummy_cols_inner_combinations = [' '.join(element) for element in dummy_cols_inner_combinations]
      
    return(dummy_cols_inner_combinations)


dummies_names = gen_dummy_col_names(data)
dummies_inner_combinations = gen_dummy_cols_inner_combinations(dummies_names)


The next step is to encode our categorical predictors, in this case `sex` and `smoker`.

In [3]:
data = pd.get_dummies(data)
data.head()

,age,bmi,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,16884.92400,1,0,0,1
1,18,33.770,1725.55230,0,1,1,0
2,28,33.000,4449.46200,0,1,1,0
3,33,22.705,21984.47061,0,1,1,0
4,32,28.880,3866.85520,0,1,1,0


## Split labels (y) and features (X) into numpy arrays

In [4]:
y_name = 'charges'
y = np.array(data[y_name])

X = data.drop(y_name, axis=1)
X_names = list(X.columns)
numeric_features = ['age', 'bmi']

X_numeric = X[numeric_features]
X_numeric
X_numeric = np.array(X_numeric)

,age,bmi
0,19,27.900
1,18,33.770
2,28,33.000
3,33,22.705
4,32,28.880
...,...,...
1333,50,30.970
1334,18,31.920
1335,18,36.850
1336,21,25.800


## Standarize the features

In [5]:
scaler = StandardScaler()
X_numeric = scaler.fit_transform(X_numeric)
X_numeric = pd.DataFrame(X_numeric, columns=numeric_features)
X_numeric

,age,bmi
0,-1.438764,-0.453320
1,-1.509965,0.509621
2,-0.797954,0.383307
3,-0.441948,-1.305531
4,-0.513149,-0.292556
...,...,...
1333,0.768473,0.050297
1334,-1.509965,0.206139
1335,-1.509965,1.014878
1336,-1.296362,-0.797813


In [6]:
categorical_features = []
for c in data.columns:
    if (c not in numeric_features) & (c != y_name):
        categorical_features.append(c)
            
X_categorical = data[categorical_features]
categorical_names =  X_categorical.columns

In [7]:
X = pd.merge(X_numeric, X_categorical, left_index=True, right_index=True)

## Interactions 
In the EDA phase, several interactions seemed to be relevant, we will for starters see how the model behave if we include up to 2nd grade interactions

In [8]:
interaction_degree = 4
interaction = PolynomialFeatures(degree=interaction_degree, include_bias=False, interaction_only=False)
X_interaction = interaction.fit_transform(X)
X_interaction_names = interaction.get_feature_names(X_names)


In [9]:
def clean_features_array(features_array, features_names,
                         categorical_features_names, dummies_inner_combinations,
                         interaction_degree):
    #this function remove non-sense terms like categorical terms n-powered
    #ie. smoker_yes^2 and inner combinations of categorical terms ie.:
    #(sex_female sex_male)
    
    # generate all expressions that if partially matched in feature names
    # will get them dropped
    non_sense_expressions = []
    for c in categorical_features_names:
        for degree in range (2, interaction_degree+1):
            expression = c + '^' + str(degree)
            non_sense_expressions.append(expression)
            
    non_sense_expressions.extend(dummies_inner_combinations)
    
    # generate a list of the columns to be dropped
    features = pd.DataFrame(features_array, columns=features_names)
    cols_to_drop = []
    for c in features.columns:
        for expression in non_sense_expressions:
            if c.find(expression) != -1:
                cols_to_drop.append(c)
    cols_to_drop = set(cols_to_drop)
    
    features.drop(cols_to_drop, axis='columns', inplace=True)
    
    features = {'names':features.columns, 'set':np.array(features)}
    
    return features
    

In [10]:
features = clean_features_array(X_interaction, X_interaction_names, categorical_names, dummies_inner_combinations, interaction_degree)

In [11]:
X_interaction = features['set']
X_interaction_names = features['names']

## Split into training and testing sets

In [12]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(X_interaction, y, test_size=0.3)

tt_sets = {'train_features':train_features, 
           'test_features':test_features,
           'train_labels':train_labels,
           'test_labels':test_labels}

for t_set in tt_sets.items():
    print (f'{t_set[0]} shape: {t_set[1].shape}')

train_features shape: (936, 78)
test_features shape: (402, 78)
train_labels shape: (936,)
test_labels shape: (402,)


In [13]:
regularization = LassoCV()
regularization.fit(train_features, train_labels)
print(f'Best alpha using built-in LassoCV: {regularization.alpha_}')
print(f'Best score using built-in LassoCV: {regularization.score(train_features, train_labels)}')
coeficients = pd.Series(regularization.coef_, index=X_interaction_names)
print(f'Lasso picked {sum(coeficients != 0)} terms and eliminated the remaining {sum(coeficients==0)} terms')

LassoCV()

Best alpha using built-in LassoCV: 115.47590247368781
Best score using built-in LassoCV: 0.8390357000878067
Lasso picked 16 terms and eliminated the remaining 62 terms


In [14]:
coefficients = coeficients[coeficients!=0]
coefficients


age                           3.082015e+03
bmi                           3.025186e+01
smoker_no                    -2.328883e+04
smoker_yes                    1.784118e-10
age^2                         1.868424e+02
bmi smoker_yes                9.873621e+03
age^3                         1.730240e+02
age^2 bmi                     1.105483e+02
bmi^2 sex_male               -2.091472e+02
bmi^2 smoker_no              -6.151825e+01
age^3 bmi                     3.003643e+01
age^3 sex_male                5.805136e+01
age^2 sex_female smoker_no    8.823763e+01
age bmi^2 sex_female          7.725934e+01
bmi^3 sex_male               -5.780944e-01
bmi^3 smoker_yes             -4.177776e+02
dtype: float64

## Dropping irrelevant interactions



In [15]:
def remove_irrevelant_features(feature_set, feature_set_column_names, features_to_keep_list):
    feature_set = pd.DataFrame(feature_set, columns=feature_set_column_names)
    cols_to_drop = []
    for c in feature_set.columns:
        if c not in features_to_keep_list:
            cols_to_drop.append(c)
    feature_set.drop(cols_to_drop, axis='columns', inplace=True)
    feature_set_names = feature_set.columns
    feature_set = np.array(feature_set)
    features = {'names':feature_set_names, 'set':feature_set}
    return features

In [16]:
train_features = remove_irrevelant_features(train_features, X_interaction_names, coefficients.index)
train_features_names = train_features['names']
train_features = train_features['set']

X_interaction_names = train_features_names

test_features = remove_irrevelant_features(test_features, X_interaction_names, coefficients.index)
test_features_names = test_features['names']
test_features = test_features['set']

### one function for both sets, would delete 3 lines of this block!!!

ValueError: Shape of passed values is (402, 78), indices imply (402, 16)

## train the model

In [ ]:
model = LinearRegression(fit_intercept = True)
model.fit(train_features, train_labels)
train_predictions = model.predict(train_features)

In [ ]:
beta_values = pd.DataFrame(model.coef_,
                          X_interaction_names,
                          columns=['coefficient'])

beta_values

## assesing performance - cross validation

In [ ]:
folds = 30
mse_crossfold = []
mse_train_values = []
mse_test_values = []
for f in range (0, folds):
    from sklearn.model_selection import train_test_split
    train_features, test_features, train_labels, test_labels = train_test_split(X_interaction, y, test_size=0.3)
    train_predictions = model.predict(train_features)
    train_errors = train_predictions - train_labels
    mse_train = (train_errors**2).mean()
    mse_train_values.append(mse_train)
    mse_train = {'type':'train','mse':mse_train}
    mse_crossfold.append(mse_train)
    test_predictions = model.predict(test_features)
    test_errors = test_predictions - test_labels
    mse_test = (test_errors**2).mean()
    mse_test_values.append(mse_test)
    mse_test = {'type':'test','mse':mse_test}
    mse_crossfold.append(mse_test)
    
mse_crossfold = pd.DataFrame(mse_crossfold)

pfig0 = px.box(mse_crossfold, x='type', y='mse',
                title='Crossfold validation, Mean Squared Error (MSE)',
                color_discrete_sequence = px.colors.qualitative.D3
            )
pfig0.show()

print(f'Train MSE mean fold values: {np.array(mse_train_values).mean()}')
print(f'Test MSE mean fold values: {np.array(mse_test_values).mean()}')

Both train and test MSE fold values show little skewness and quite similar medians. As expected, the test MSE fold values show a larger dispersion do to the fact that these sets have less values and therefore larger magnitude square errorrs are harder to compensate when calculating the mean.

From the above statement we could say that the crossfold validation was successful, but if we really want to be obnoxious about it, since we would be dealing with the mean of means and we have a large enough (n=100) set of observations, we could appeal to the central limit theorem (the mean of means follow a normal distribution) and perform a two sided hypothesis testing to show that the means of the MSE crossfold values aren't statistically different.

In [ ]:

displot = plt.figure(figsize=(12.8,8.16))
title = f'distribution of MSE: crossfold validation'
fig = sns.displot(data=mse_crossfold, x='mse', hue='type', kind='kde', palette='muted')
fig.set(title=title)
file_name = title + ' ' + datetime.now().isoformat()[:19]
fig.savefig(file_name, bbox_inches='tight')
plt.figure()



### Aspin-Welch Unequal-Variance T-Test
[reference](https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Two-Sample_T-Test_from_Means_and_SDs.pdf)

- null: mean of the MSE of train and test are the same
- alternate : mean of the MSE of train and test are different

In [ ]:
aspin_welch_result = stats.ttest_ind(mse_train_values, mse_test_values, axis=0, equal_var=False, nan_policy='omit')

In [ ]:
aspin_welch_result

With a significance level of alpha=0.05, since this is a 2 sided test, the observed pvalue should be either lower than 0.025 or higher than 0.975 to reject the null hypothesis, with an observed pvalue of 0.5935 there is no staistical evidence to reject it. In other words, there is no evidence to make us think that the means of MSE fold values of the trainning and testing sets are different.

# Inferential statistics assumptions
1. **Linearity**: It is assumed that the relationship between each predictor variable and the criterion variable is linear. 
    If this assumption is not met, then the predictions may systematically overestimate the actual values for one range of values on a predictor variable and underestimate them for another (bias).
    
    While working with high-dimensional data, it may not be practical to plot every dimension vs the prediction. An alternative is to use a prediction error plot, as it lets visualize how well the model does compared to the truth.


2. **Residuals are normaly distributed**. The residuals (aka. erros) are the difference between predictions and the real values of the labels found in the data set.



3. **Homoscedasticity**: Variances of the residuals are the same for all predicted values.

Even though moderate violations of Assumptions 1 to 3 do not present a serious threat for the significance of predictor variables, even small transgressions to them could compromise the validity on certain predictions.

In [ ]:
X_interaction = pd.DataFrame(X_interaction, columns=X_interaction_names)
y = pd.DataFrame(data[y_name])
data = pd.merge(y, X_interaction, left_index=True, right_index=True)
data


In [ ]:
def standarize_arr(_array):
    _arr_mean = _array.mean()
    _arr_stdev = _array.std()
    _normalized_arr = (_array - _arr_mean)/_arr_stdev
    return _normalized_arr

def create_error_analysis_df(_data, _y_name, _train_labels, _train_predictions, _test_labels, _test_predictions):
    _error_analysis_df_train = pd.DataFrame()
    _error_analysis_df_train[_y_name] = _train_labels
    _error_analysis_df_train['prediction'] = _train_predictions
    _error_analysis_df_train['split'] = 'train'
    
    _error_analysis_df_test = pd.DataFrame()
    _error_analysis_df_test[_y_name] = _test_labels
    _error_analysis_df_test['prediction'] = _test_predictions
    _error_analysis_df_test['split'] = 'test'
    
    _error_analysis_df = pd.concat([_error_analysis_df_train, _error_analysis_df_test], ignore_index=True)
    _error_analysis_df['residual'] = _error_analysis_df['prediction'] -  _error_analysis_df[y_name]
    _error_analysis_df['standarized_residual'] = standarize_arr(_error_analysis_df['residual'])
    
    _error_analysis_df['residual_theoretical_normal_P'] = stats.norm.cdf(_error_analysis_df['standarized_residual'])
    _error_analysis_df['residual_observed_P'] = _error_analysis_df['residual'].rank(pct = True) 
          
    return(_error_analysis_df)


In [ ]:
error_data = create_error_analysis_df(data, y_name, train_labels, train_predictions, test_labels, 
test_predictions)

In [ ]:
InteractiveShell.ast_node_interactivity = 'last'

In [ ]:
fig1 = px.scatter(error_data,
                 x = y_name,
                 y = 'prediction',
                 marginal_x = 'histogram',
                 marginal_y = 'histogram',
                 color = 'split',
                 title = 'Linearity: Prediction error plot',
                 color_discrete_sequence = px.colors.qualitative.D3
               )

fig1.update_traces(histnorm='probability', selector={'type':'histogram'})

fig1.add_shape(type = 'line',
              line = {'dash' : 'dash'},
              x0 = y.min(), y0=y.min(),
              x1 = y.max(), y1=y.max()
              )

fig1.update_layout(xaxis = {'scaleanchor':'y', 'scaleratio':1, 'ticks':'outside'},
                   yaxis = {'ticks':'outside'},
                   autosize = False,
                   width = 500,
                   height = 500,
                   dragmode = False
                  )
fig1.show()

In [ ]:
fig2 = px.scatter(error_data,
                 x = error_data['residual_theoretical_normal_P'],
                 y = error_data['residual_observed_P']     ,
                 color_discrete_sequence = px.colors.qualitative.D3          
               )

fig2.add_shape(type = 'line',
              line = {'dash' : 'dash'},
              x0 = 0, y0 = 0,
              x1 = 1, y1 = 1
              )

fig2.update_layout(xaxis = {'scaleanchor':'y', 'scaleratio':1, 'ticks':'outside'},
                   yaxis = {'ticks':'outside'},
                   autosize = False,
                   width = 500,
                   height = 500,
                   dragmode = False
                  )
fig2.show()

In [ ]:
fig3 = px.scatter(error_data,
                 x = 'prediction',
                 y = 'standarized_residual',
                 color = 'split',
                 title = 'Residuals Homoscedasticity',
                 color_discrete_sequence = px.colors.qualitative.D3
               )

fig3.show()